In [2]:
!pip install tqdm_joblib

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tqdm_joblib: filename=tqdm_joblib-0.0.3-py3-none-any.whl size=1637 sha256=3ce7f2a807addde69042472878f50f457f497d1eca3c3b11a74b666430074046
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\47\f1\7e\4489c9ac0b95389801c1a739e5d92492299536b070a1f688a0
Successfully built tqdm_joblib
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import sqlite3
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import re
import numpy
from joblib import Parallel, delayed
tqdm.pandas()
from tqdm_joblib import tqdm_joblib

C:\Users\Admin\Documents\Python Scripts\lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\subtitles.csv",nrows=500)

In [9]:
df.head()

,num,name,subtitles,chunked_data
0,9180533,the.message.(1976).eng.1cd,Watch any video online with OpenSUBTITLESFre...,watch video online opensubtitlesfree browser e...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,Ah Theres PrincessDawn and Terry with the B...,ah princessdawn terry blooney looney soldiersp...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,iYumis Cells i iEpisode Extremely Polite Yu...,iyumis cell iepisode extremely polite yumii iy...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,Watch any video online with OpenSUBTITLESFre...,watch video online opensubtitlesfree browser e...
4,9180600,broker.(2022).eng.1cd,Watch any video online with OpenSUBTITLESFre...,watch video online opensubtitlesfree browser e...


In [10]:
df.drop(['chunked_data'],axis=1,inplace=True)

In [41]:
df

,num,name,subtitle_content,subtitle_content_encoded
0,9180533,the message (1976) eng 1cd,Watch any video online with OpenSUBTITLESFre...,"[0.024978460744023323, 0.05731463432312012, -0..."
0,9180533,the message (1976) eng 1cd,ia Was it like this when John the Baptistcame...,"[-0.007689794059842825, 0.14881308376789093, -..."
0,9180533,the message (1976) eng 1cd,Muhammadas a mercy to mankind The Scholars an...,"[-0.019260983914136887, 0.16961021721363068, -..."
0,9180533,the message (1976) eng 1cd,ark agesi iand everywhere the oldcivilization...,"[0.038837723433971405, 0.14701123535633087, -0..."
0,9180533,the message (1976) eng 1cd,f Abrahami ithe Kaaba had nowbecome a house o...,"[0.04441261291503906, 0.11914205551147461, 0.0..."
...,...,...,...,...
499,9183658,effects (1979) eng 1cd,Oh come here you bitch Groans Come here Th...,"[-0.012349319644272327, -0.04526472091674805, ..."
499,9183658,effects (1979) eng 1cd,out this Everything is fine nobodyis hurt nob...,"[0.05480166897177696, -0.0013620925601571798, ..."
499,9183658,effects (1979) eng 1cd,I didnt slash anybody Celeste You think some...,"[-0.03947385773062706, -0.03519144281744957, 0..."
499,9183658,effects (1979) eng 1cd,tling music engine starts You really killed...,"[-0.04161975160241127, 0.018621189519762993, 0..."


In [11]:
df.shape

(500, 3)

In [12]:
def srt_files_to_chuncks(row, window_size=512, overlapping=100):
    chunks = []
    i = 0
    while i < len(row['subtitles']):
        chunk_end = min(i + window_size, len(row['subtitles']))
        chunks.append(row['subtitles'][i:chunk_end])
        i += window_size - overlapping
    return chunks

def process_row(row):
    return srt_files_to_chuncks(row)


In [13]:
num_cores = -1  # Using all available CPU cores
with tqdm(total=len(df)) as pbar:
    processed_rows = Parallel(n_jobs=num_cores)(
        delayed(process_row)(row) for index, row in df.iterrows()
    )
    pbar.update()

df['subtitle_content'] = processed_rows

  0%|▏                                                                               | 1/500 [00:07<1:06:24,  7.98s/it]


In [14]:
print(df['subtitle_content'][0][:100])

['  Watch any video online with OpenSUBTITLESFree Browser extension osdblinkext  In the name of God the most gracious the most Merciful  From Muhammad the Messenger of God  to Heraclius the emperor of Byzantium  greetings to him who is thefollower of righteous guidance  I bid you to hear the divine call  I am the messenger of God to the people  accept Islam for your salvation  He speaks of a new prophet in Arabia  Was it like this when John the Baptistcame to king Herod  out of the desert crying about salvati', 'ia  Was it like this when John the Baptistcame to king Herod  out of the desert crying about salvation  To Muqawqis Patriarch of Alexandria  Kisra emperor of Persia  Muhammad calls you with the call of God  Accept Islam for your salvation  embrace Islam  You come out of the desertsmelling of camel and goat  To tell Persia where he should kneel  Muhammad Messenger of God  Who gave him this authority  God sent Muhammadas a mercy to mankind  The Scholars and Historians of Islam Th

In [15]:
 df['name'] = df['name'].progress_apply(lambda x: re.sub('\.', ' ', x))

100%|████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<?, ?it/s]


In [16]:
df.head()

,num,name,subtitles,subtitle_content
0,9180533,the message (1976) eng 1cd,Watch any video online with OpenSUBTITLESFre...,[ Watch any video online with OpenSUBTITLESFr...
1,9180583,here comes the grump s01 e09 joltin jack in bo...,Ah Theres PrincessDawn and Terry with the B...,[ Ah Theres PrincessDawn and Terry with the ...
2,9180592,yumis cells s02 e13 episode 2 13 (2022) eng 1cd,iYumis Cells i iEpisode Extremely Polite Yu...,[ iYumis Cells i iEpisode Extremely Polite Y...
3,9180594,yumis cells s02 e14 episode 2 14 (2022) eng 1cd,Watch any video online with OpenSUBTITLESFre...,[ Watch any video online with OpenSUBTITLESFr...
4,9180600,broker (2022) eng 1cd,Watch any video online with OpenSUBTITLESFre...,[ Watch any video online with OpenSUBTITLESFr...


In [17]:
with tqdm(total=1) as pbar:
    df = df.explode('subtitle_content', ignore_index=False)
    pbar.update()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.00it/s]


In [18]:
df.shape

(32493, 4)

In [19]:
df.drop(['subtitles'],axis=1,inplace=True)

In [21]:
!pip install chromadb

     -------------------------------------- 526.8/526.8 kB 8.3 MB/s eta 0:00:00
  Using cached kubernetes-29.0.0-py2.py3-none-any.whl (1.6 MB)
     ---------------------------------------- 47.2/47.2 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 92.0/92.0 kB 1.0 MB/s eta 0:00:00
     -------------------------------------- 106.1/106.1 kB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 3.8 MB/s eta 0:00:00
  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.24.0 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.19.1 which is incompatible.


In [23]:
!pip install sentence_transformers

     -------------------------------------- 171.5/171.5 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 9.0/9.0 MB 4.0 MB/s eta 0:00:00
     -------------------------------------- 287.4/287.4 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


In [24]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer,util

In [27]:
client = chromadb.PersistentClient(path=r"D:\search-engine.db")

In [28]:
client.heartbeat()

1715238169097237300

In [29]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Admin\Documents\Python Scripts\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
collection = client.create_collection(
        name="subtitles_embedded3",
        metadata={"hnsw:space": "cosine"},
        embedding_function=sentence_transformer_ef
    )

In [31]:
df_for_em = df

In [32]:
df_for_em.shape

(32493, 3)

In [33]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [34]:
df_with_em = df_for_em 

In [35]:
from joblib import parallel_backend

In [36]:
def encoding_content(x):
    return model.encode(x, normalize_embeddings=True).tolist()

# df_with_em.loc[:, 'subtitle_content_encoded'] = df_for_em['subtitle_content'].progress_apply(encoding_content)

In [ ]:
num_cores = -1
with parallel_backend('threading', n_jobs=num_cores):
    encoded_contents = Parallel()(delayed(encoding_content)(content) for content in tqdm(df_for_em['subtitle_content']))

df_with_em['subtitle_content_encoded'] = encoded_contents

 39%|████████████████████████████▎                                           | 12768/32493 [1:37:19<2:01:15,  2.71it/s]

In [28]:
df_with_em.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32493 entries, 0 to 499
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   num                       32493 non-null  int64 
 1   name                      32493 non-null  object
 2   subtitle_content          32493 non-null  object
 3   subtitle_content_encoded  32493 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [29]:
df_with_em.head()

,num,name,subtitle_content,subtitle_content_encoded
0,9180533,the message (1976) eng 1cd,Watch any video online with OpenSUBTITLESFre...,"[0.024978460744023323, 0.05731463432312012, -0..."
0,9180533,the message (1976) eng 1cd,ia Was it like this when John the Baptistcame...,"[-0.007689794059842825, 0.14881308376789093, -..."
0,9180533,the message (1976) eng 1cd,Muhammadas a mercy to mankind The Scholars an...,"[-0.019260983914136887, 0.16961021721363068, -..."
0,9180533,the message (1976) eng 1cd,ark agesi iand everywhere the oldcivilization...,"[0.038837723433971405, 0.14701123535633087, -0..."
0,9180533,the message (1976) eng 1cd,f Abrahami ithe Kaaba had nowbecome a house o...,"[0.04441261291503906, 0.11914205551147461, 0.0..."


In [31]:
for i in tqdm(range(df_with_em.shape[0])):
    collection.add(
        documents=df_with_em['subtitle_content'].iloc[i],
        embeddings=df_with_em['subtitle_content_encoded'].iloc[i],
        metadatas= {'subtitle_name': df_with_em['name'].iloc[i], 'num': str(df_with_em['num'].iloc[i])},
        ids=str(i)
    )

100%|███████████████████████████████████████████████████████████████████████████| 32493/32493 [04:23<00:00, 123.43it/s]


In [32]:
import random
temp = df_with_em.iloc[random.randint(0, len(df_with_em))]
for i in range(4):
    if i == 3:
        continue
    else:
        print(temp[i])

9182996
i touched all your stuff (2014) eng 1cd
ouched  Shell be pleasedwhen she arrives  muffled laughing  This is the note left by Luke  Said Would it kill youto get a stepladder  PS I touched all your stuff  Im feeling the love yeah  the bright shinysilvery love  Every TV show CNN  every news organizationis calling  every radio show is calling  They just went crazy  And it was kind of a lotto deal with  Im trying to work as wellbeinghaving been gone  for a couple of weeksfrom work  trying to catch up on that  At the same timenow remember  Ive got this


C:\Users\ravin\AppData\Local\Temp\ipykernel_8272\2583520840.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(temp[i])


In [34]:
collection = client.get_collection(name="subtitles_embedded3", embedding_function=sentence_transformer_ef)

In [35]:
def clean_text(text):
    text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\r\n', '', text)
    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'watch any video online with opensubtitles free browser extension osdblinkext', '', text)
    text = text.strip()
    return text
# new_df['subtitle_content'] = new_df['subtitle_content'].apply(clean_text)

In [36]:
# query = "ross you know yesterday he asked me to fix him up with somebody"
query = "hey ross how are you"

query = clean_text(query)

In [37]:
query_em = encoding_content(query)

In [38]:
search_results = collection.query(query_embeddings=query_em, n_results=5)

In [39]:
print("Top 10 Results")
for i, res in enumerate(search_results['metadatas'][0]):
    print(f"Result {i+1} : ", res['subtitle_name']) 


Top 5 Results
Result 1 :  better call saul s06 e10 nippy (2022) eng 1cd
Result 2 :  survivor s14 e15 survivor fiji the reunion (2007) eng 1cd
Result 3 :  bleed american (2019) eng 1cd
Result 4 :  the great food truck race s05 e07 finale at mile 0 winning keys in key west (2014) eng 1cd
Result 5 :  the great food truck race s05 e07 finale at mile 0 winning keys in key west (2014) eng 1cd


In [ ]:
load_dotenv()

headers = {
    "accept": "application/json",
    "Authorization": os.getenv("API_KEY")
}


In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer,util
client = chromadb.PersistentClient(path=r"D:\search_engine_subtitles_files")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [ ]:
collection = client.create_collection(
        name="subtitles_embedded3",
        metadata={"hnsw:space": "cosine"},
        embedding_function=sentence_transformer_ef
    )
model = SentenceTransformer("all-MiniLM-L6-v2")